## First class Functions
Functions in python are first-class objects. <br><br>
First-class objects are program entities which can be:
- created at runtime
- assigned to a variable or element in a data structure
- passed as an argument to a function
- returned as a result of a function
<br><br>
Part III of the book focuses on the implications and practical applications of treating functions as objects.

### Treating a function like an object

In [4]:
# Check function type

def factorial(n):
    '''returns n!'''
    return 1 if n<2 else n*factorial(n-1)

print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


In [5]:
fact = factorial
print(fact)
print(fact(5))
print(map(factorial,range(11)))
list(map(fact,range(11)))


<function factorial at 0x000002143B812480>
120


[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

### Higher order functions
A higher order function is a function which takes a function as an argument or returns a function as the result. <br>
The BIF _sorted_ is an example of higher order function.

In [8]:
# Sorted is a higher order function

fruits = ['strawberry', 'fig', 'apple', 'cherry']
print(sorted(fruits, key=len))
fruits

['fig', 'apple', 'cherry', 'strawberry']


['strawberry', 'fig', 'apple', 'cherry']

In [9]:
# Sorting a list of words by their reversed spelling

def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse)

['apple', 'fig', 'strawberry', 'cherry']

### Modern replacement for map, filter, reduce
These are some of the most common higher order functions, which are offered by most of the functional programming languages, but better alternatives are available for most of their use cases. <br>
In fact a listcomp or genexp does the same job of map and filter combined, but are more readable. <br>
Map and filter return a generator, their sub

In [11]:
# Map and filter VS list comprehensions

list(map(fact,range(6)))
print([fact(n) for n in range(6)])
print(list(map(factorial, filter(lambda n: n%2, range(6)))))
[factorial(n) for n in range(6) if n % 2]

[1, 1, 2, 6, 24, 120]
[1, 6, 120]


[1, 6, 120]

For the functools.reduce function, in the most common use case (summation), it is better served by the sum BIF 

In [13]:
# Sum VS reduce

from functools import reduce
from operator import add

print(reduce(add, range(100)))
sum(range(100))

4950


4950

Other reducing BIF are _all_ and _any_

### Anonymous Functions (lambda)
To use a higher order function, sometimes it is convenient to create a small one-off function --> lambda. <br>
The lambda keyword creates an anonymous function within a python expression. <br>
The body of lambda is limited to be a pure expression. Thus outside of the context of arguments to higher order functions, lambdas are rarely used in Python. <br>
Lambda is syntactic sugar, but it is a normal function object in python, just like any other function.

In [14]:
# Example lambda

fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry']
sorted(fruits, key=lambda word: word[::-1])

['apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### The seven flavors of callable objects
These are the callable objects in python:
- user-defined functions
- built-in functions
- built-in methods
- methods
- classes
- class instances
- generator functions (the ones using the yield keyword)
<br><br>
To determine weather an object is a callable use the BIF callable()

#### User defined callable types
This can be useful when you want to create a function-like object that have some internal state which must be kept across invocations. <br>
This approach can be useful for defining decorators which need to "remember" something between calls of the decorator.

In [15]:
# Building class instances which work as callable objects

import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()


bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
callable(bingo)

0
2


True

### Function introspection
Runtime introspection of functions as objects.

In [16]:
# Function objects have many attributes
dir(factorial)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']

In [ ]:
# Assign arbitrary attributes to functions

def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj.last_name)).upper()
upper_case_name.short_description = 'Customer name'

In [17]:
# Attributes of functions which don't exist in plain instances

class C: pass
obj = C()
def func(): pass
sorted(set(dir(func))-set(dir(obj)))

['__annotations__',
 '__builtins__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__',
 '__type_params__']

In [19]:
# Keyword-only argument

def tag(name, *content, cls=None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value) for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name)for c in content)
    else:
        return '<%s%s />' % (name, attr_str)
    

print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name="img"))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}
print(tag(**my_tag))

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />
<img src="sunset.jpg" title="Sunset Boulevard" />


To specify keyword only arguments when defining a function, name them after the argument prefixed with *.

In [23]:
def f(a, *, b):
    return a, b
print(f(1, b=2))
f(a=1, b=2)

(1, 2)


(1, 2)

### Retrieving informations about parameters

In [24]:
def clip(text, max_len = 80):
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >=0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()


print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

(80,)
<code object clip at 0x000002143B0C89F0, file "C:\Users\giacomo.mascher\AppData\Local\Temp\ipykernel_22260\1931583144.py", line 1>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [25]:
# Using the inspect module is easier

from inspect import signature

sig = signature(clip)
print(sig)
str(sig)
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

(text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [26]:
import inspect

sig = inspect.signature(tag)
my_tag = {'name':'img', 'title':'Sunset Boulevard', 'src':'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
print(bound_args)
for name, value in bound_args.arguments.items():
    print(name, '=', value)
del my_tag['name']
bound_args = sig.bind(**my_tag)

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>
name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


TypeError: missing a required argument: 'name'

### Function annotations
It allows to attach metadata to the parameters of a function declaration and its return value. <br>
No processing is done with annotations, they are just stored in the annotations attribute of a function. Annotations have no meaning to the python interpreter.

In [27]:
# Annotations example

def my_func(text: str, max_len: int = 80) -> str:
    return 'ciao'

my_func.__annotations__

{'text': str, 'max_len': int, 'return': str}

In [28]:
# inspect.signature is useful to extract annotations
from inspect import signature

sig = signature(my_func)
print(sig.return_annotation)
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)
    

<class 'str'>
<class 'str'> : text = <class 'inspect._empty'>
<class 'int'> : max_len = 80


### Packages for functional programming
A list of packages from the PSL that support functional programming.

#### The operator module
To have other functions similar to the BIF sum.

In [29]:
# Factorial without using recursion

from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

In [30]:
# Remove lambda using the operator module

from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

In [12]:
# Itemgetter to sort a list of tuples

metro_data = [
    ('Tokyo', 'JP', 36.9, (35.7, 139.6)),
    ('Delhi NRC', 'IN', 21.9, (28.6, 77.2)),
    
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Delhi NRC', 'IN', 21.9, (28.6, 77.2))
('Tokyo', 'JP', 36.9, (35.7, 139.6))


In [13]:
# Demo of attrgetter
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]
print(metro_areas[0])

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

Metropolis(name='Tokyo', cc='JP', pop=36.9, coord=LatLong(lat=35.7, long=139.6))
('Delhi NRC', 28.6)
('Tokyo', 35.7)


Freezing arguments with functools.partial, functools.partialmethod does the same but with methods.

In [15]:
# Freezing arguments with functools.partial
from operator import mul
from functools import partial

triple = partial(mul, 3)
print(triple(7))
print(list(map(triple, range(1,10))))
print(triple.func)
print(triple.args)

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]
<built-in function mul>
(3,)
